# 1. Prepare feature matrix

Feature matrix are saved via previous program. So in this part, they are loaded from local files.

# 2. Run in Spark

In [37]:
spark = SparkSession.builder.appName("HomeDepot").getOrCreate()
df = spark.read.csv("/Users/jiemin/Documents/cmu/tasks/task14/train_s_formatted.csv",header=True,inferSchema=True)

In [38]:
df.take(1)

[Row(id=2, product_uid=100001, relevance=3.0, cosine_1=0.276609, cosine_2=0.206489, cosine_3=0.0, common_word=1.0, fuzzy_train=56.0, jaccard_train=0.2, query_length=2.0)]

In [39]:
trainSet, testSet = df.randomSplit([0.8, 0.2])

In [41]:
trainSet.take(1)

[Row(id=2, product_uid=100001, relevance=3.0, cosine_1=0.276609, cosine_2=0.206489, cosine_3=0.0, common_word=1.0, fuzzy_train=56.0, jaccard_train=0.2, query_length=2.0)]

## (1) Linear regression

In [42]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.functions import udf
vs = VectorAssembler(inputCols=["cosine_1","cosine_2","cosine_3","common_word","fuzzy_train","jaccard_train","query_length"],outputCol='features')
train_lr = vs.transform(trainSet)

In [43]:
train_lr.take(1)

[Row(id=2, product_uid=100001, relevance=3.0, cosine_1=0.276609, cosine_2=0.206489, cosine_3=0.0, common_word=1.0, fuzzy_train=56.0, jaccard_train=0.2, query_length=2.0, features=DenseVector([0.2766, 0.2065, 0.0, 1.0, 56.0, 0.2, 2.0]))]

In [44]:
lr = LinearRegression(featuresCol='features', regParam=0.3, elasticNetParam=0.8,labelCol='relevance',maxIter=1000)
model = lr.fit(train_lr)

In [45]:
lr_summary = model.summary
print("Coefficients: %s" % str(model.coefficients))
print("Intercept: %s" % str(model.intercept))
print("numIterations: %d" % lr_summary.totalIterations)
print("objectiveHistory: %s" % str(lr_summary.objectiveHistory))
print("RMSE: %f" % lr_summary.rootMeanSquaredError)
print("r2: %f" % lr_summary.r2)

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 2.38163736952
numIterations: 1
objectiveHistory: [0.5]
RMSE: 0.532482
r2: -0.000000


In [46]:
test_lr = vs.transform(testSet)
predict_lr = model.transform(test_lr)

In [47]:
predict_lr.show(5)

+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+------------------+
| id|product_uid|relevance|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|            features|        prediction|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+------------------+
| 16|     100005|     2.33|0.145886|0.383327|  0.0451|        1.0|       55.0|     0.066667|         3.0|[0.145886,0.38332...|2.3816373695162474|
| 18|     100006|      3.0|0.224883|0.428235|0.086006|        1.0|       83.0|       0.0625|         2.0|[0.224883,0.42823...|2.3816373695162474|
| 27|     100009|      3.0|0.391382|0.365989|0.139376|        2.0|      100.0|     0.166667|         2.0|[0.391382,0.36598...|2.3816373695162474|
|101|     100019|     1.33|0.146573|0.509738|0.051765|        1.0|       60.0|     0.076923|         2.0|[0.146573,0.50973..

In [50]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="relevance", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predict_lr)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse)

Root Mean Squared Error (RMSE) on validation data = 0.539858


## (2) Random forest

In [51]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer 
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml import Pipeline 

rf = RandomForestRegressor(featuresCol="features")
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(train_lr)
labelIndexer = StringIndexer(inputCol="relevance", outputCol="label").fit(train_lr)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
model = pipeline.fit(train_lr)
predict_rf = model.transform(test_lr)

In [69]:
train_lr.show(20)

+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+
| id|product_uid|relevance|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|            features|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+
|  2|     100001|      3.0|0.276609|0.206489|     0.0|        1.0|       56.0|          0.2|         2.0|[0.276609,0.20648...|
|  3|     100001|      2.5|     0.0|0.206489|     0.0|        0.0|       19.0|          0.0|         2.0|(7,[1,4,6],[0.206...|
|  9|     100002|      3.0|     0.0|0.240663|0.056452|        0.0|       19.0|          0.0|         2.0|[0.0,0.240663,0.0...|
| 17|     100005|     2.67|0.428689|0.383327|0.132527|        3.0|      100.0|     0.230769|         3.0|[0.428689,0.38332...|
| 20|     100006|     2.67| 0.33732|0.428235|0.115759|        2.0|       82.0|        0.125|         3.0|[0.337

In [53]:
predict_rf.show(5)

+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+-----+--------------------+------------------+
| id|product_uid|relevance|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|            features|label|     indexedFeatures|        prediction|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+-----+--------------------+------------------+
| 16|     100005|     2.33|0.145886|0.383327|  0.0451|        1.0|       55.0|     0.066667|         3.0|[0.145886,0.38332...|  1.0|[0.145886,0.38332...|2.3808204811518245|
| 18|     100006|      3.0|0.224883|0.428235|0.086006|        1.0|       83.0|       0.0625|         2.0|[0.224883,0.42823...|  0.0|[0.224883,0.42823...|1.6434343093608743|
| 27|     100009|      3.0|0.391382|0.365989|0.139376|        2.0|      100.0|     0.166667|         2.0|[0.391382,0.36598...|  0.0|[0.

In [54]:
# evluator for regression 
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predict_rf)
print("Root Mean Squared Error = %g" % rmse)

Root Mean Squared Error = 1.5277


## (3) Output result

In [56]:
df_test = spark.read.csv("/Users/jiemin/Documents/cmu/tasks/task14/test_s_formatted.csv",header=True,inferSchema=True)

In [57]:
df.show(5)

+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+
| id|product_uid|relevance|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+
|  2|     100001|      3.0|0.276609|0.206489|     0.0|        1.0|       56.0|          0.2|         2.0|
|  3|     100001|      2.5|     0.0|0.206489|     0.0|        0.0|       19.0|          0.0|         2.0|
|  9|     100002|      3.0|     0.0|0.240663|0.056452|        0.0|       19.0|          0.0|         2.0|
| 16|     100005|     2.33|0.145886|0.383327|  0.0451|        1.0|       55.0|     0.066667|         3.0|
| 17|     100005|     2.67|0.428689|0.383327|0.132527|        3.0|      100.0|     0.230769|         3.0|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+
only showing top 5 rows



In [62]:
df_train = vs.transform(df)
df_train.show(5)

+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+
| id|product_uid|relevance|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|            features|
+---+-----------+---------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+
|  2|     100001|      3.0|0.276609|0.206489|     0.0|        1.0|       56.0|          0.2|         2.0|[0.276609,0.20648...|
|  3|     100001|      2.5|     0.0|0.206489|     0.0|        0.0|       19.0|          0.0|         2.0|(7,[1,4,6],[0.206...|
|  9|     100002|      3.0|     0.0|0.240663|0.056452|        0.0|       19.0|          0.0|         2.0|[0.0,0.240663,0.0...|
| 16|     100005|     2.33|0.145886|0.383327|  0.0451|        1.0|       55.0|     0.066667|         3.0|[0.145886,0.38332...|
| 17|     100005|     2.67|0.428689|0.383327|0.132527|        3.0|      100.0|     0.230769|         3.0|[0.428

### 1) Linear regression

In [59]:
lr = LinearRegression(featuresCol='features', regParam=0.3, elasticNetParam=0.8,labelCol='relevance',maxIter=1000)
model_lr = lr.fit(df_train)
test = vs.transform(df_test)
predict_linear = model_lr.transform(test)

In [63]:
p_lr = predict_linear
p_lr = p_lr.drop("product_uid","cosine_1","cosine_2","cosine_3","common_word","fuzzy_train","jaccard_train","query_length","features")
p_lr.toPandas().to_csv("predict_result_lr",index=False)

### 2) Random forest

In [65]:
model_rf = pipeline.fit(df_train)
predict_random = model.transform(test)

In [67]:
predict_random.show(5)

+---+-----------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+--------------------+------------------+
| id|product_uid|cosine_1|cosine_2|cosine_3|common_word|fuzzy_train|jaccard_train|query_length|            features|     indexedFeatures|        prediction|
+---+-----------+--------+--------+--------+-----------+-----------+-------------+------------+--------------------+--------------------+------------------+
|  1|     100001|     0.0|0.206489|0.043806|        0.0|       28.0|          0.0|         3.0|[0.0,0.206489,0.0...|[0.0,0.206489,0.0...| 2.498930155903658|
|  4|     100001|     0.0|0.206489|     0.0|        0.0|       24.0|          0.0|         3.0|(7,[1,4,6],[0.206...|(7,[1,4,6],[0.206...|2.6292240267005305|
|  5|     100001|0.300237|0.206489|0.064453|        1.0|       61.0|     0.166667|         3.0|[0.300237,0.20648...|[0.300237,0.20648...|2.0852016514611664|
|  6|     100001|0.485935|0.206489|0.136106|        1.0|  

In [72]:
p_rf = predict_random
p_rf = p_rf.drop("product_uid","cosine_1","cosine_2","cosine_3","common_word","fuzzy_train","jaccard_train","query_length","features")
# p_rf.toPandas().to_csv("predict_result_rf",index=False)
p_rf.show(100000)

Py4JJavaError: An error occurred while calling o1460.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 138.0 failed 1 times, most recent failure: Lost task 1.0 in stage 138.0 (TID 211, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$11: (vector) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$25.apply(RDD.scala:830)
	at org.apache.spark.rdd.RDD$$ano